In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, RuntimeEncoder
from qiskit.visualization import plot_histogram
import numpy as np
from QGates_4_qubits import *
from magnetization import *
import matplotlib.pyplot as plt
import json

API_KEY = "INSERT YOUR API KEY"
n = 4
lamb = 0.5
lamb_tag = "0_5"
T = 2
delta_t = T/200
Mz_exact, t_axis_exact = get_data_Mz(T,delta_t,lamb)
t_axis = np.array([])
Mz = np.array([])
stdv_Mz = np.array([])
stdv_mean_Mz = np.array([])

#Conncetion to the remote quantum computer
service = QiskitRuntimeService(channel="ibm_quantum_platform",token=API_KEY)
print("Service established")
backend = service.backend("ibm_sherbrooke")
print("Connected to:",backend.name)
sampler = Sampler(mode=backend)
pubs = []       #List of the quantum circuits run

for i in range(int(len(t_axis_exact)/10)):
    t_axis = np.append(t_axis,t_axis_exact[i*10])

#Construction of the 20 quantum circuits to perform the simulation
for t in t_axis:
    qc = QuantumCircuit(n)
    #Preparation of the qubits in the state |1111>
    for i in range(n):
        qc.x(i)
    #Disentangling process
    apply_Udis(qc,n,lamb)
    #Application of the evolution operator
    ks = [0,2,1,-1]
    for i in range(n):
        E = get_E(n,ks[i],lamb)
        qc.p(-1*E*t,i)
    #Recalculation of the state in the original basis
    apply_Udis_inv(qc,n,lamb)    
    qc.measure_all()

    layout = range(n)
    qc_transpiled = transpile(qc,backend,initial_layout=layout,optimization_level=2)
    pubs.append(qc_transpiled)

qc_transpiled.draw("mpl")

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-07-01 14:45:22,730: Default instance not set. Searching all available instances.


Service established


qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-07-01 14:45:24,728: Default instance not set. Searching all available instances.


QiskitBackendNotFoundError: 'No backend matches the criteria. Learn more about available backends here https://quantum.cloud.ibm.com/docs/en/guides/qpu-information#view-your-resources'

In [ ]:
#Generation and storage of the results
print("Backend:",sampler.backend())
qc_job = sampler.run(pubs, shots=2**10)

with open("results/result_lambda_"+lamb_tag+".json", "w") as file:
    json.dump(qc_job.result(), file, cls=RuntimeEncoder)

f = open("results/times_lambda_"+lamb_tag+".txt", "w")
for t in t_axis:
    f.write(str(t)+"\n")
f.close()
print("Task completed")